# DLE303 Assessment 3 - Main
## Rachel Tucker & Michael Cuffe

## Dependencies:
- Python 3.12

In [154]:
%%capture
!pip install transformers
!pip install torch
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install seaborn

# import libraries

In [155]:
import transformers
from transformers import trainer
from torch.nn import CrossEntropyLoss
from transformers import get_scheduler
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from tqdm import tqdm
from accelerate.test_utils.scripts.test_distributed_data_loop import test_data_loader
import torch
import torch.optim as optim
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import pandas as pd
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
import torch.nn.functional as F

In [156]:
# Ignore warnings after first viewing
import warnings
warnings.filterwarnings(action='once')

# This is where we split the data into training and testing sets 

In [157]:
# Load the dataset
data = pd.read_csv('data/200k_Short_Texts.csv')
data = data.dropna()
data = data.sample(frac=1).reset_index(drop=True)

# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Tokenize the Data

In [158]:
tokenizer = BertTokenizer.from_pretrained('huawei-noah/TinyBERT_General_4L_312D')

def tokenize_data(data):
    return tokenizer(
        data['text'].tolist(),
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors='pt',
    )

train_encodings = tokenize_data(train_data)
test_encodings = tokenize_data(test_data)

C:\Python312\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


# Create a Dataset class

In [159]:
class HumorDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx]).clone().detach()
        return item

    def __len__(self):
        return len(self.labels)


# Flatten\ing (Kind of)

In [160]:
train_labels = train_data['humor'].astype(int).tolist()
test_labels = test_data['humor'].astype(int).tolist()

train_dataset = HumorDataset(train_encodings, train_labels)
test_dataset = HumorDataset(test_encodings, test_labels)

# Define the Model

In [161]:
model = BertForSequenceClassification.from_pretrained('huawei-noah/TinyBERT_General_4L_312D', num_labels=2);

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at huawei-noah/TinyBERT_General_4L_312D and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Initial Training

In [162]:
def compute_metrics(p):
    preds = p.predictions.argmax(-1)
    labels = p.label_ids
    f1 = f1_score(labels, preds, average='weighted')
    accuracy = accuracy_score(labels, preds)
    precision = precision_score(labels, preds, average='weighted')
    sensitivity = recall_score(labels, preds, average='weighted')
    
    # Calculate BCE score
    labels_tensor = torch.tensor(labels, dtype=torch.float32)
    preds_tensor = torch.tensor(preds, dtype=torch.float32)
    bce_score = F.binary_cross_entropy_with_logits(preds_tensor, labels_tensor).item()
    
    return {
        "f1": f1,
        "accuracy": accuracy,
        "precision": precision,
        "sensitivity": sensitivity,
        "bce_score": bce_score
    }

In [163]:
# Pre optimised training
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=10,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=600,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=6000,
    eval_strategy="steps",  # Evaluate every `logging_steps`
    eval_steps=6000,  # Evaluation and logging will be done every 100 steps
    save_steps=10000,
    save_total_limit=2,  # Only last 2 models are saved
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

trainer.train()

Step,Training Loss,Validation Loss,F1,Accuracy,Precision,Sensitivity,Bce Score
6000,0.138100,0.162544,0.965593,0.965625,0.967139,0.965625,0.517668
12000,0.083600,0.071258,0.978546,0.978550,0.978824,0.978550,0.513191
18000,0.065000,0.082225,0.981350,0.981350,0.981351,0.981350,0.513142
24000,0.050700,0.095385,0.980274,0.980275,0.980305,0.980275,0.513292
30000,0.046700,0.093048,0.982050,0.982050,0.982073,0.982050,0.513297
36000,0.030800,0.103203,0.980948,0.980950,0.981077,0.980950,0.512453
42000,0.028200,0.122142,0.981425,0.981425,0.981429,0.981425,0.513354
48000,0.020700,0.107447,0.982475,0.982475,0.982478,0.982475,0.512523
54000,0.016600,0.143328,0.979473,0.979475,0.979593,0.979475,0.513224
60000,0.013600,0.124525,0.981124,0.981125,0.981169,0.981125,0.512765


TrainOutput(global_step=100000, training_loss=0.03149638834953308, metrics={'train_runtime': 2990.9713, 'train_samples_per_second': 534.943, 'train_steps_per_second': 33.434, 'total_flos': 2598943065600000.0, 'train_loss': 0.03149638834953308, 'epoch': 10.0})

# Fine Tuning the Model

In [164]:
# Use the torch.optim.AdamW optimizer
optimizer = optim.AdamW(model.parameters(), lr=5e-5)

In [165]:
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=16)

In [166]:
# Use the get_scheduler function to create a linear scheduler
num_training_steps = len(train_dataloader) * 3
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

In [167]:
# Define the loss function
loss_fn = CrossEntropyLoss()

# First Attempt at Optimizing

In [168]:
# Implement the tuning
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

epochs = 3
for epoch in range(epochs):
    model.train()
    for batch in tqdm(train_dataloader):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()

100%|██████████| 10000/10000 [05:24<00:00, 30.81it/s]


In [169]:
# Evaluate the model
from sklearn.metrics import accuracy_score

model.eval()
predictions, true_labels = [], []
for batch in test_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)
    logits = outputs.logits
    preds = torch.argmax(logits, dim=1)  # Convert logits to predicted labels  
    predictions.append(preds)
    true_labels.append(batch['labels'])

In [170]:
# Convert lists of tensors to single tensors
predictions = torch.cat(predictions, dim=0)
true_labels = torch.cat(true_labels, dim=0)
# Flatten the tensors
predictions = predictions.view(-1).cpu().numpy()
true_labels = true_labels.view(-1).cpu().numpy()

In [171]:
# Assuming `true_labels` and `predictions` are already defined
f1 = f1_score(true_labels, predictions, average='weighted')
accuracy = accuracy_score(true_labels, predictions)
precision = precision_score(true_labels, predictions, average='weighted')
sensitivity = recall_score(true_labels, predictions, average='weighted')

# Calculate BCE score
true_labels_tensor = torch.tensor(true_labels, dtype=torch.float32)
predictions_tensor = torch.tensor(predictions, dtype=torch.float32)
bce_score = F.binary_cross_entropy_with_logits(predictions_tensor, true_labels_tensor).item()

# Print the metrics
print("F1 Score:", f1)
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Sensitivity: {sensitivity}")
print(f"BCE Score: {bce_score}")

F1 Score: 0.9813994907041109
Accuracy: 0.9814
Precision: 0.9814258553816416
Sensitivity: 0.9814
BCE Score: 0.5127627849578857


# Second Attempt at Optimizing

In [172]:
# Implement the tuning
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

epochs =10
for epoch in range(epochs):
    model.train()
    for batch in tqdm(train_dataloader):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()

100%|██████████| 10000/10000 [03:53<00:00, 42.89it/s]


In [173]:
# Evaluate the model
from sklearn.metrics import accuracy_score

model.eval()
predictions, true_labels = [], []
for batch in test_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)
    logits = outputs.logits
    preds = torch.argmax(logits, dim=1)  # Convert logits to predicted labels  
    predictions.append(preds)
    true_labels.append(batch['labels'])

In [174]:
# Convert lists of tensors to single tensors
predictions = torch.cat(predictions, dim=0)
true_labels = torch.cat(true_labels, dim=0)
# Flatten the tensors
predictions = predictions.view(-1).cpu().numpy()
true_labels = true_labels.view(-1).cpu().numpy()

In [175]:
# Assuming `true_labels` and `predictions` are already defined
f1 = f1_score(true_labels, predictions, average='weighted')
accuracy = accuracy_score(true_labels, predictions)
precision = precision_score(true_labels, predictions, average='weighted')
sensitivity = recall_score(true_labels, predictions, average='weighted')

# Calculate BCE score
true_labels_tensor = torch.tensor(true_labels, dtype=torch.float32)
predictions_tensor = torch.tensor(predictions, dtype=torch.float32)
bce_score = F.binary_cross_entropy_with_logits(predictions_tensor, true_labels_tensor).item()

# Print the metrics
print(f"F1 Score: {f1}")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Sensitivity: {sensitivity}")
print(f"BCE Score: {bce_score}")

F1 Score: 0.9813994907041109
Accuracy: 0.9814
Precision: 0.9814258553816416
Sensitivity: 0.9814
BCE Score: 0.5127627849578857


In [185]:
# Tokenize the input sentence
input_sentence = "NLP 303 Assessment 3 Report was incredibly difficult."
inputs = tokenizer(input_sentence, return_tensors='pt', padding=True, truncation=True, max_length=512)

# Move the inputs to the appropriate device
inputs = {key: val.to(device) for key, val in inputs.items()}

# Put the model in evaluation mode
model.eval()

# Get the model's predictions
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    probabilities = F.softmax(logits, dim=1)

# Get the probability of the text being humorous
humorous_prob = probabilities[0][1].item()
print(f"Probability of text being humorous: {humorous_prob:.4f}")

Probability of text being humorous: 0.0333


In [177]:
# Tokenize the input sentence
input_sentence = "Two fish in a tank, one turns to the other and says; how the hell do you drive this thing?"
inputs = tokenizer(input_sentence, return_tensors='pt', padding=True, truncation=True, max_length=512)

# Move the inputs to the appropriate device
inputs = {key: val.to(device) for key, val in inputs.items()}

# Put the model in evaluation mode
model.eval()

# Get the model's predictions
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    probabilities = F.softmax(logits, dim=1)

# Get the probability of the text being humorous
humorous_prob = probabilities[0][1].item()
print(f"Probability of text being humorous: {humorous_prob:.4f}")

Probability of text being humorous: 1.0000


In [178]:
#eof